# RSF AGENCY SKILL TEST

## DATA CALCULATION AND CLEANING PIPELINE

This pipeline is used to clean the data from the original data set:
- Part 1. Sales Data.csv
- Part 1. WoW Row Data.csv
- Part 2. Advanced Chartgraphs.csv

`By José Daniel Rojas`

## 1. LIBRARIES

In [345]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


## 2. DATA SOURCES

In [346]:
# Data Load

wow_data = pd.read_csv(r"C:\Users\joey_\Documents\Visual Code (Clone)\Proyectos\AM3 TEST\00_test_A\00_datasources\00_original\Part 1. WoW Row Data.csv", sep=';', thousands=',', decimal='.')
sales_data = pd.read_csv(r"C:\Users\joey_\Documents\Visual Code (Clone)\Proyectos\AM3 TEST\00_test_A\00_datasources\00_original\Part 1. Sales Data.csv", sep=';', thousands=',', decimal='.')
advanced_data = pd.read_csv(r"C:\Users\joey_\Documents\Visual Code (Clone)\Proyectos\AM3 TEST\00_test_A\00_datasources\00_original\Part 2. Advanced Chartgraphs.csv", sep=';', thousands=',', decimal='.')

## 3. DATA CLEANING AND TRANSFORMATIONS

## WoW Row Data pipeline
(Looks like a fact table grouped by week in August) 

In [347]:
# Dropping rows with NaN values in 'CHATTER' column
wow_data.dropna(subset=['CHATTER'], inplace=True)

# Filling 'Hours' columns with 0
wow_data['Hours'] = wow_data['Hours'].fillna(0)

#Transform 'NetSales' column to numeric
wow_data['NetSales'] = wow_data['NetSales'].replace({'\$': '', ',': ''}, regex=True).astype(float)
wow_data['NetSales'] = wow_data['NetSales'].fillna(0)

#Transform 'Sales/HR' column to numeric
wow_data['Sales/HR'] = wow_data['Sales/HR'].replace({'\$': '', ',': ''}, regex=True).astype(float)
wow_data['Sales/HR'] = wow_data['Sales/HR'].fillna(0)

#Transform 'TargetSalesHR' column to numeric
wow_data['TargetSalesHR'] = wow_data['TargetSalesHR'].replace({'\$': '', ',': ''}, regex=True).astype(float)
wow_data['TargetSalesHR'] = wow_data['TargetSalesHR'].fillna(0)

#Transform 'TargetSPH' column to numeric
wow_data['TargetSPH'] = wow_data['TargetSPH'].replace({'\$': '', ',': ''}, regex=True).astype(float)
wow_data['TargetSPH'] = wow_data['TargetSPH'].fillna(0)

#Transform 'TargetNetSales per WK' column to numeric
wow_data['TargetNetSales per WK'] = wow_data['TargetNetSales per WK'].replace({'\$': '', ',': ''}, regex=True).astype(float)
wow_data['TargetNetSales per WK'] = wow_data['TargetNetSales per WK'].fillna(0)

# Create Column Month Text to Numeric
wow_data['Month (Number)'] = wow_data['Month'].map({
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
})
wow_data['Month (Number)'] = wow_data['Month (Number)'].fillna(0)
wow_data['Month (Number)'] = wow_data['Month (Number)'].astype(int)

#Creating column 'Week (Number)' from 'Week'
wow_data['Week (Number)'] = wow_data['Week'].str.extract('(\d+)').astype(int)

# Creating key CHATTER-MODEL-TENURE-WEEK
wow_data['CHATTER-MODEL-TENURE'] = wow_data['CHATTER'] + '-' + wow_data['Model'] + '-' + wow_data['Tenure'] + '-' + wow_data['Week (Number)'].astype(str)

wow_data.head()

,CHATTER,Model,Tenure,NetSales,Hours,Sales/HR,TargetSPH,TargetNetSales per WK,TargetSalesHR,Month,Week,Month (Number),Week (Number),CHATTER-MODEL-TENURE
0,JESBONI,MODEL 5,Training,457.54,12.0,38.13,28.0,8400.0,336.0,August,Week 1,8,1,JESBONI-MODEL 5-Training-1
1,KALVIN,MODEL 5,Training,25.58,4.0,6.40,28.0,8400.0,112.0,August,Week 1,8,1,KALVIN-MODEL 5-Training-1
2,KALVIN,MODEL 8,Training,23.20,4.0,5.80,23.0,6650.0,92.0,August,Week 1,8,1,KALVIN-MODEL 8-Training-1
3,KALVIN,MODEL 12,Training,303.20,4.0,75.80,39.0,11900.0,156.0,August,Week 1,8,1,KALVIN-MODEL 12-Training-1
4,JOHN C,MODEL 5,Training,69.60,4.0,17.40,28.0,8400.0,112.0,August,Week 1,8,1,JOHN C-MODEL 5-Training-1


In [348]:
### QUALITY REPORT (POST FIRST CLEANING)

def create_data_quality_report(df):
    # Initialize dictionary to store quality metrics
    quality_report = {
        'Column_Name': [],
        'Data_Type': [],
        'Total_Rows': [],
        'Missing_Values': [],
        'Missing_Percentage': [],
        'Blank_Values': [],
        'Blank_Percentage': [],
        'Zero_Values': [],
        'Zero_Percentage': [],
        'Duplicate_Values': [],
        'Unique_Values': []
    }
    
    total_rows = len(df)
    
    for column in df.columns:
        # Count missing values
        missing_count = df[column].isna().sum()
        
        # Count blank values (empty strings)
        blank_count = df[column].astype(str).str.strip().eq('').sum()
        
        # Count zero values for numeric columns
        if pd.api.types.is_numeric_dtype(df[column]):
            zero_count = (df[column] == 0).sum()
        else:
            zero_count = 0
            
        # Count unique values
        unique_count = df[column].nunique()
        
        # Add to report
        quality_report['Column_Name'].append(column)
        quality_report['Data_Type'].append(str(df[column].dtype))
        quality_report['Total_Rows'].append(total_rows)
        quality_report['Missing_Values'].append(missing_count)
        quality_report['Missing_Percentage'].append(round(missing_count/total_rows * 100, 2))
        quality_report['Blank_Values'].append(blank_count)
        quality_report['Blank_Percentage'].append(round(blank_count/total_rows * 100, 2))
        quality_report['Zero_Values'].append(zero_count)
        quality_report['Zero_Percentage'].append(round(zero_count/total_rows * 100, 2))
        quality_report['Duplicate_Values'].append(df[column].duplicated().sum())
        quality_report['Unique_Values'].append(unique_count)
    
    # Create DataFrame from the quality report
    quality_df = pd.DataFrame(quality_report)
    
    # Sort by missing percentage (descending)
    quality_df = quality_df.sort_values('Missing_Percentage', ascending=False)
    
    return quality_df

# Create the quality report
quality_report = create_data_quality_report(wow_data)

selected_columns = wow_data.columns[0:14].tolist() 
Q_Report = quality_report[quality_report['Column_Name'].isin(selected_columns)]
Q_Report

,Column_Name,Data_Type,Total_Rows,Missing_Values,Missing_Percentage,Blank_Values,Blank_Percentage,Zero_Values,Zero_Percentage,Duplicate_Values,Unique_Values
0,CHATTER,object,663,0,0.0,0,0.0,0,0.00,604,59
1,Model,object,663,0,0.0,0,0.0,0,0.00,649,14
2,Tenure,object,663,0,0.0,0,0.0,0,0.00,660,3
3,NetSales,float64,663,0,0.0,0,0.0,52,7.84,122,541
4,Hours,float64,663,0,0.0,0,0.0,32,4.83,611,52
5,Sales/HR,float64,663,0,0.0,0,0.0,84,12.67,135,528
6,TargetSPH,float64,663,0,0.0,0,0.0,0,0.00,633,30
7,TargetNetSales per WK,float64,663,0,0.0,0,0.0,18,2.71,651,12
8,TargetSalesHR,float64,663,0,0.0,0,0.0,32,4.83,398,265
9,Month,object,663,0,0.0,0,0.0,0,0.00,662,1


In [349]:
# Duplicated Values on CHATTER-MODEL-TENURE
duplicated_values = wow_data[wow_data.duplicated(subset=['CHATTER-MODEL-TENURE'])]
duplicated_values 

,CHATTER,Model,Tenure,NetSales,Hours,Sales/HR,TargetSPH,TargetNetSales per WK,TargetSalesHR,Month,Week,Month (Number),Week (Number),CHATTER-MODEL-TENURE
541,MERICK,MODEL 13,Training,8.8,8.0,1.1,7.0,2100.0,56.0,August,Week 4,8,4,MERICK-MODEL 13-Training-4


In [350]:
# Resolve duplicates in 'CHATTER-MODEL-TENURE', keeping record with non-zero NetSales
wow_data = wow_data.sort_values(by=['CHATTER-MODEL-TENURE', 'NetSales'], ascending=[True, False])
wow_data = wow_data.drop_duplicates(subset=['CHATTER-MODEL-TENURE'], keep='first')

In [351]:
# Duplicated Values on CHATTER-MODEL-TENURE
duplicated_values = wow_data[wow_data.duplicated(subset=['CHATTER-MODEL-TENURE'])]
duplicated_values 

,CHATTER,Model,Tenure,NetSales,Hours,Sales/HR,TargetSPH,TargetNetSales per WK,TargetSalesHR,Month,Week,Month (Number),Week (Number),CHATTER-MODEL-TENURE


In [352]:
### QUALITY REPORT (POST SECOND CLEANING)

def create_data_quality_report(df):
    # Initialize dictionary to store quality metrics
    quality_report = {
        'Column_Name': [],
        'Data_Type': [],
        'Total_Rows': [],
        'Missing_Values': [],
        'Missing_Percentage': [],
        'Blank_Values': [],
        'Blank_Percentage': [],
        'Zero_Values': [],
        'Zero_Percentage': [],
        'Duplicate_Values': [],
        'Unique_Values': []
    }
    
    total_rows = len(df)
    
    for column in df.columns:
        # Count missing values
        missing_count = df[column].isna().sum()
        
        # Count blank values (empty strings)
        blank_count = df[column].astype(str).str.strip().eq('').sum()
        
        # Count zero values for numeric columns
        if pd.api.types.is_numeric_dtype(df[column]):
            zero_count = (df[column] == 0).sum()
        else:
            zero_count = 0
            
        # Count unique values
        unique_count = df[column].nunique()
        
        # Add to report
        quality_report['Column_Name'].append(column)
        quality_report['Data_Type'].append(str(df[column].dtype))
        quality_report['Total_Rows'].append(total_rows)
        quality_report['Missing_Values'].append(missing_count)
        quality_report['Missing_Percentage'].append(round(missing_count/total_rows * 100, 2))
        quality_report['Blank_Values'].append(blank_count)
        quality_report['Blank_Percentage'].append(round(blank_count/total_rows * 100, 2))
        quality_report['Zero_Values'].append(zero_count)
        quality_report['Zero_Percentage'].append(round(zero_count/total_rows * 100, 2))
        quality_report['Duplicate_Values'].append(df[column].duplicated().sum())
        quality_report['Unique_Values'].append(unique_count)
    
    # Create DataFrame from the quality report
    quality_df = pd.DataFrame(quality_report)
    
    # Sort by missing percentage (descending)
    quality_df = quality_df.sort_values('Missing_Percentage', ascending=False)
    
    return quality_df

# Create the quality report
quality_report = create_data_quality_report(wow_data)

selected_columns = wow_data.columns[0:14].tolist() 
Q_Report = quality_report[quality_report['Column_Name'].isin(selected_columns)]
Q_Report

,Column_Name,Data_Type,Total_Rows,Missing_Values,Missing_Percentage,Blank_Values,Blank_Percentage,Zero_Values,Zero_Percentage,Duplicate_Values,Unique_Values
0,CHATTER,object,662,0,0.0,0,0.0,0,0.00,603,59
1,Model,object,662,0,0.0,0,0.0,0,0.00,648,14
2,Tenure,object,662,0,0.0,0,0.0,0,0.00,659,3
3,NetSales,float64,662,0,0.0,0,0.0,51,7.70,121,541
4,Hours,float64,662,0,0.0,0,0.0,32,4.83,610,52
5,Sales/HR,float64,662,0,0.0,0,0.0,83,12.54,134,528
6,TargetSPH,float64,662,0,0.0,0,0.0,0,0.00,632,30
7,TargetNetSales per WK,float64,662,0,0.0,0,0.0,18,2.72,650,12
8,TargetSalesHR,float64,662,0,0.0,0,0.0,32,4.83,397,265
9,Month,object,662,0,0.0,0,0.0,0,0.00,661,1


### Findings:

- The data looks like a fact table grouped WoW for August 2024.
- There is no further information on the data related to the year.
- Only the chatter "Merick" presented two records for the same key combination `CHATTER-MODEL-WEEK`.
- Significant zero values found in numeric columns: `NetSales` (7.84% zeros), `Hours` (4.83% zeros), `Sales/HR` (12.67% zeros), `TargetNetSales per WK` (2.71% zeros), and `TargetSalesHR` (4.83% zeros), indicating potential missing or unreported data.These null were managed with 0.00 values.
- The dataset is limited to August only, with no data from other months, potentially restricting temporal analysis.
- The duplicate record for `MERICK-MODEL 13-Training-4` was identified and resolved taking as a reference the record with the `NetSales` value.
- Numeric conversions (e.g., `NetSales`, `Sales/HR`) use `astype(float)` without error handling. 

In [353]:
### Sending cleaning data to the folder '01_cleaned_data'

wow_data.to_csv(r"C:\Users\joey_\Documents\Visual Code (Clone)\Proyectos\AM3 TEST\00_test_A\00_datasources\01_cleaned_data\wow_data.csv", index=False)
print("Done")

Done


## Sales Data Pipeline

In [354]:
# Dropping nulls

sales_data.dropna(subset=['Chatter Name'], inplace=True)

# Parsing Date & Time in Sales Data
sales_data['Date & time'] = pd.to_datetime(sales_data['Date & time'], format='%d/%m/%Y %H:%M')
sales_data['Date'] = sales_data['Date & time'].dt.date
sales_data['Time'] = sales_data['Date & time'].dt.time
sales_data['Week'] = sales_data['Date & time'].dt.isocalendar().week

#Transform 'Earnings' column to numeric
sales_data['Earnings'] = sales_data['Earnings'].replace({'\$': '', ',': ''}, regex=True).astype(float)
sales_data['Earnings'] = sales_data['Earnings'].fillna(0)

#Transform 'Commissions' from string to percentage or float
sales_data['Commission'] = sales_data['Commission'].replace('%', '', regex=True).astype(float)/100



sales_data.head()



,Chatter Name,Date & time,Fan,MODEL,Type,Earnings,Commission,Payout,Date,Time,Week
0,ANGELICA,2024-08-12 23:56:00,Wes Campbell,MODEL 9,Messages,36.0,0.05,NaN,2024-08-12,23:56:00,33
1,ANGELICA,2024-08-12 23:54:00,J,MODEL 9,Messages,20.0,0.05,NaN,2024-08-12,23:54:00,33
2,ANGELICA,2024-08-12 23:53:00,J,MODEL 9,Messages,12.0,0.05,NaN,2024-08-12,23:53:00,33
3,ANGELICA,2024-08-12 23:47:00,J,MODEL 9,Messages,13.6,0.05,NaN,2024-08-12,23:47:00,33
4,ANGELICA,2024-08-12 23:33:00,Wes Campbell,MODEL 9,Messages,17.6,0.05,NaN,2024-08-12,23:33:00,33


In [355]:
### QUALITY REPORT (POST FIRST CLEANING)

def create_data_quality_report(df):
    # Initialize dictionary to store quality metrics
    quality_report = {
        'Column_Name': [],
        'Data_Type': [],
        'Total_Rows': [],
        'Missing_Values': [],
        'Missing_Percentage': [],
        'Blank_Values': [],
        'Blank_Percentage': [],
        'Zero_Values': [],
        'Zero_Percentage': [],
        'Duplicate_Values': [],
        'Unique_Values': []
    }
    
    total_rows = len(df)
    
    for column in df.columns:
        # Count missing values
        missing_count = df[column].isna().sum()
        
        # Count blank values (empty strings)
        blank_count = df[column].astype(str).str.strip().eq('').sum()
        
        # Count zero values for numeric columns
        if pd.api.types.is_numeric_dtype(df[column]):
            zero_count = (df[column] == 0).sum()
        else:
            zero_count = 0
            
        # Count unique values
        unique_count = df[column].nunique()
        
        # Add to report
        quality_report['Column_Name'].append(column)
        quality_report['Data_Type'].append(str(df[column].dtype))
        quality_report['Total_Rows'].append(total_rows)
        quality_report['Missing_Values'].append(missing_count)
        quality_report['Missing_Percentage'].append(round(missing_count/total_rows * 100, 2))
        quality_report['Blank_Values'].append(blank_count)
        quality_report['Blank_Percentage'].append(round(blank_count/total_rows * 100, 2))
        quality_report['Zero_Values'].append(zero_count)
        quality_report['Zero_Percentage'].append(round(zero_count/total_rows * 100, 2))
        quality_report['Duplicate_Values'].append(df[column].duplicated().sum())
        quality_report['Unique_Values'].append(unique_count)
    
    # Create DataFrame from the quality report
    quality_df = pd.DataFrame(quality_report)
    
    # Sort by missing percentage (descending)
    quality_df = quality_df.sort_values('Missing_Percentage', ascending=False)
    
    return quality_df

# Create the quality report
quality_report = create_data_quality_report(sales_data)

selected_columns = sales_data.columns[0:14].tolist() 
Q_Report = quality_report[quality_report['Column_Name'].isin(selected_columns)]
Q_Report

,Column_Name,Data_Type,Total_Rows,Missing_Values,Missing_Percentage,Blank_Values,Blank_Percentage,Zero_Values,Zero_Percentage,Duplicate_Values,Unique_Values
7,Payout,float64,4535,4535,100.00,0,0.0,0,0.0,4534,0
2,Fan,object,4535,21,0.46,0,0.0,0,0.0,2941,1593
6,Commission,float64,4535,20,0.44,0,0.0,0,0.0,4527,7
0,Chatter Name,object,4535,0,0.00,0,0.0,0,0.0,4488,47
1,Date & time,datetime64[ns],4535,0,0.00,0,0.0,0,0.0,937,3598
3,MODEL,object,4535,0,0.00,0,0.0,0,0.0,4523,12
4,Type,object,4535,0,0.00,0,0.0,0,0.0,4533,2
5,Earnings,float64,4535,0,0.00,0,0.0,0,0.0,4133,402
8,Date,object,4535,0,0.00,0,0.0,0,0.0,4528,7
9,Time,object,4535,0,0.00,0,0.0,0,0.0,3174,1361


In [356]:
# Transform 'Fan' nulls to "Unknown"
sales_data['Fan'] = sales_data['Fan'].fillna('Unknown')
sales_data['Fan'] = sales_data['Fan'].astype(str)

# Replace 'Fan' values of "Unknow" with "Unknown"
sales_data['Fan'] = sales_data['Fan'].replace('Unknow', 'Unknown')

# Replace null 'Commission' values with 0 for trainee chatters
sales_data['Commission'] = sales_data['Commission'].fillna(0)

# Calculate 'Payout' as Earnings * Commission, rounded to 2 decimal places
sales_data['Payout'] = (sales_data['Earnings'] * sales_data['Commission']).round(2)
sales_data['Payout'] = sales_data['Payout'].astype(float)


In [357]:
### QUALITY REPORT (POST SECOND CLEANING)

def create_data_quality_report(df):
    # Initialize dictionary to store quality metrics
    quality_report = {
        'Column_Name': [],
        'Data_Type': [],
        'Total_Rows': [],
        'Missing_Values': [],
        'Missing_Percentage': [],
        'Blank_Values': [],
        'Blank_Percentage': [],
        'Zero_Values': [],
        'Zero_Percentage': [],
        'Duplicate_Values': [],
        'Unique_Values': []
    }
    
    total_rows = len(df)
    
    for column in df.columns:
        # Count missing values
        missing_count = df[column].isna().sum()
        
        # Count blank values (empty strings)
        blank_count = df[column].astype(str).str.strip().eq('').sum()
        
        # Count zero values for numeric columns
        if pd.api.types.is_numeric_dtype(df[column]):
            zero_count = (df[column] == 0).sum()
        else:
            zero_count = 0
            
        # Count unique values
        unique_count = df[column].nunique()
        
        # Add to report
        quality_report['Column_Name'].append(column)
        quality_report['Data_Type'].append(str(df[column].dtype))
        quality_report['Total_Rows'].append(total_rows)
        quality_report['Missing_Values'].append(missing_count)
        quality_report['Missing_Percentage'].append(round(missing_count/total_rows * 100, 2))
        quality_report['Blank_Values'].append(blank_count)
        quality_report['Blank_Percentage'].append(round(blank_count/total_rows * 100, 2))
        quality_report['Zero_Values'].append(zero_count)
        quality_report['Zero_Percentage'].append(round(zero_count/total_rows * 100, 2))
        quality_report['Duplicate_Values'].append(df[column].duplicated().sum())
        quality_report['Unique_Values'].append(unique_count)
    
    # Create DataFrame from the quality report
    quality_df = pd.DataFrame(quality_report)
    
    # Sort by missing percentage (descending)
    quality_df = quality_df.sort_values('Missing_Percentage', ascending=False)
    
    return quality_df

# Create the quality report
quality_report = create_data_quality_report(sales_data)

selected_columns = sales_data.columns[0:14].tolist() 
Q_Report = quality_report[quality_report['Column_Name'].isin(selected_columns)]
Q_Report

,Column_Name,Data_Type,Total_Rows,Missing_Values,Missing_Percentage,Blank_Values,Blank_Percentage,Zero_Values,Zero_Percentage,Duplicate_Values,Unique_Values
0,Chatter Name,object,4535,0,0.0,0,0.0,0,0.00,4488,47
1,Date & time,datetime64[ns],4535,0,0.0,0,0.0,0,0.00,937,3598
2,Fan,object,4535,0,0.0,0,0.0,0,0.00,2943,1592
3,MODEL,object,4535,0,0.0,0,0.0,0,0.00,4523,12
4,Type,object,4535,0,0.0,0,0.0,0,0.00,4533,2
5,Earnings,float64,4535,0,0.0,0,0.0,0,0.00,4133,402
6,Commission,float64,4535,0,0.0,0,0.0,20,0.44,4527,8
7,Payout,float64,4535,0,0.0,0,0.0,20,0.44,4245,290
8,Date,object,4535,0,0.0,0,0.0,0,0.00,4528,7
9,Time,object,4535,0,0.0,0,0.0,0,0.00,3174,1361


In [358]:
#Filtering the trainees chatters who contains "TRAINING/"
sales_data = sales_data[~sales_data['Chatter Name'].str.contains("TRAINING/")]  

### Findings:

- The data is a fact table with sales by chatters during week 33 of 2024 (2024-08-12 to 2024-08-18) .
- Were found 21 records in null and where replaced with "Unknown".
- Was found a fan called as "Unknow" and replaced with "Unknown".
- Where found 20 'Commission' in null and replaced with 0, these null are related to a trainee chatters "Merick" and "Richards".
- The payouts where calculated with the formula: `sales_data['Earnings'] * sales_data['Commission']`
- The dataset may lack year information, similar to `wow_data`, which could restrict temporal analysis.
- Finally, Trainees chatters were removed from the dataset as was required.

In [359]:
sales_data.head()

,Chatter Name,Date & time,Fan,MODEL,Type,Earnings,Commission,Payout,Date,Time,Week
0,ANGELICA,2024-08-12 23:56:00,Wes Campbell,MODEL 9,Messages,36.0,0.05,1.80,2024-08-12,23:56:00,33
1,ANGELICA,2024-08-12 23:54:00,J,MODEL 9,Messages,20.0,0.05,1.00,2024-08-12,23:54:00,33
2,ANGELICA,2024-08-12 23:53:00,J,MODEL 9,Messages,12.0,0.05,0.60,2024-08-12,23:53:00,33
3,ANGELICA,2024-08-12 23:47:00,J,MODEL 9,Messages,13.6,0.05,0.68,2024-08-12,23:47:00,33
4,ANGELICA,2024-08-12 23:33:00,Wes Campbell,MODEL 9,Messages,17.6,0.05,0.88,2024-08-12,23:33:00,33


In [360]:
### Sending cleaning data to the folder '01_cleaned_data'

sales_data.to_csv(r"C:\Users\joey_\Documents\Visual Code (Clone)\Proyectos\AM3 TEST\00_test_A\00_datasources\01_cleaned_data\sales_data.csv", index=False)
print("Done")

Done


## ADVANCED CHARTGRAPH

In [361]:
# Dropping nulls

sales_data.dropna(subset=['Chatter Name'], inplace=True)

# Parsing Date & Time in Sales Data
sales_data['Date & time'] = pd.to_datetime(sales_data['Date & time'], format='%d/%m/%Y %H:%M')
sales_data['Date'] = sales_data['Date & time'].dt.date
sales_data['Time'] = sales_data['Date & time'].dt.time
sales_data['Week'] = sales_data['Date & time'].dt.isocalendar().week

#Transform 'Earnings' column to numeric
sales_data['Earnings'] = sales_data['Earnings'].replace({'\$': '', ',': ''}, regex=True).astype(float)
sales_data['Earnings'] = sales_data['Earnings'].fillna(0)

#Transform 'Commissions' from string to percentage or float
sales_data['Commission'] = sales_data['Commission'].replace('%', '', regex=True).astype(float)/100



sales_data.head()



,Chatter Name,Date & time,Fan,MODEL,Type,Earnings,Commission,Payout,Date,Time,Week
0,ANGELICA,2024-08-12 23:56:00,Wes Campbell,MODEL 9,Messages,36.0,0.0005,1.80,2024-08-12,23:56:00,33
1,ANGELICA,2024-08-12 23:54:00,J,MODEL 9,Messages,20.0,0.0005,1.00,2024-08-12,23:54:00,33
2,ANGELICA,2024-08-12 23:53:00,J,MODEL 9,Messages,12.0,0.0005,0.60,2024-08-12,23:53:00,33
3,ANGELICA,2024-08-12 23:47:00,J,MODEL 9,Messages,13.6,0.0005,0.68,2024-08-12,23:47:00,33
4,ANGELICA,2024-08-12 23:33:00,Wes Campbell,MODEL 9,Messages,17.6,0.0005,0.88,2024-08-12,23:33:00,33


In [362]:
# Dropping nulls
advanced_data.dropna(subset=['Date'], inplace=True)

# Parsing 'Date' in Advanced Data
advanced_data['Date'] = pd.to_datetime(advanced_data['Date'], format='%Y-%m-%d').dt.date

# Parsing 'Last Edited Time' in Advanced Data
advanced_data['Last Edited Time'] = pd.to_datetime(advanced_data['Last Edited Time'], format='%d/%m/%Y %H:%M')

# Calculate Start and End of Week (week starts on Sunday)
advanced_data['Start of Week'] = pd.to_datetime(advanced_data['Date']) - pd.offsets.Week(weekday=6)
advanced_data['End of Week'] = pd.to_datetime(advanced_data['Date']) + pd.offsets.Week(weekday=6) - pd.Timedelta(days=1)

# Convert to date (remove time component)
advanced_data['Start of Week'] = advanced_data['Start of Week'].dt.date
advanced_data['End of Week'] = advanced_data['End of Week'].dt.date

# Check available columns
print("Columns in advanced_data:", advanced_data.columns.tolist())

#Rename ' Revenue ' column to 'Revenue'
advanced_data.rename(columns={' Revenue ': 'Revenue'}, inplace=True)

# Transform 'Revenue' column to numeric
advanced_data['Revenue'] = advanced_data['Revenue'].replace({r'\$': '', ',': ''}, regex=True).str.strip()
advanced_data['Revenue'] = pd.to_numeric(advanced_data['Revenue'], errors='coerce').fillna(0)

#Drop columns 'Week' and 'Month'
advanced_data.drop(['Week', 'Month'], axis=1, inplace=True)

advanced_data.head()

Columns in advanced_data: ['Date', 'Boost Group', 'DM Times', 'Name', 'Fans Gained', ' Revenue ', 'Agency', 'Last Edited Time', 'Week', 'Month', 'Half', 'Start of Week', 'End of Week']


,Date,Boost Group,DM Times,Name,Fans Gained,Revenue,Agency,Last Edited Time,Half,Start of Week,End of Week
0,2024-10-01,"Group A,Group B",Double Boost (8PM & 2AM),MODEL 18,738.0,1331.24,Agency N,2024-10-17 11:42:00,First Half,2024-09-29,2024-10-05
1,2024-10-01,"Group A,Group B",Double Boost (8PM & 2AM),MODEL 20,980.0,2260.13,Agency R,2024-10-17 11:56:00,First Half,2024-09-29,2024-10-05
2,2024-10-02,Group B,8AM & 2PM,MODEL 21,404.0,4833.27,Agency T,2024-10-17 11:59:00,First Half,2024-09-29,2024-10-05
3,2024-10-02,Group A,8AM & 2PM,MODEL 1,575.0,1552.59,Agency T,2024-10-17 09:08:00,First Half,2024-09-29,2024-10-05
4,2024-10-02,"Group A,Group B",Double Boost (8PM & 2AM),MODEL 6,728.0,7591.88,Agency R,2024-10-17 10:26:00,First Half,2024-09-29,2024-10-05


In [363]:
### QUALITY REPORT (POST FIRST CLEANING)

def create_data_quality_report(df):
    # Initialize dictionary to store quality metrics
    quality_report = {
        'Column_Name': [],
        'Data_Type': [],
        'Total_Rows': [],
        'Missing_Values': [],
        'Missing_Percentage': [],
        'Blank_Values': [],
        'Blank_Percentage': [],
        'Zero_Values': [],
        'Zero_Percentage': [],
        'Duplicate_Values': [],
        'Unique_Values': []
    }
    
    total_rows = len(df)
    
    for column in df.columns:
        # Count missing values
        missing_count = df[column].isna().sum()
        
        # Count blank values (empty strings)
        blank_count = df[column].astype(str).str.strip().eq('').sum()
        
        # Count zero values for numeric columns
        if pd.api.types.is_numeric_dtype(df[column]):
            zero_count = (df[column] == 0).sum()
        else:
            zero_count = 0
            
        # Count unique values
        unique_count = df[column].nunique()
        
        # Add to report
        quality_report['Column_Name'].append(column)
        quality_report['Data_Type'].append(str(df[column].dtype))
        quality_report['Total_Rows'].append(total_rows)
        quality_report['Missing_Values'].append(missing_count)
        quality_report['Missing_Percentage'].append(round(missing_count/total_rows * 100, 2))
        quality_report['Blank_Values'].append(blank_count)
        quality_report['Blank_Percentage'].append(round(blank_count/total_rows * 100, 2))
        quality_report['Zero_Values'].append(zero_count)
        quality_report['Zero_Percentage'].append(round(zero_count/total_rows * 100, 2))
        quality_report['Duplicate_Values'].append(df[column].duplicated().sum())
        quality_report['Unique_Values'].append(unique_count)
    
    # Create DataFrame from the quality report
    quality_df = pd.DataFrame(quality_report)
    
    # Sort by missing percentage (descending)
    quality_df = quality_df.sort_values('Missing_Percentage', ascending=False)
    
    return quality_df

# Create the quality report
quality_report = create_data_quality_report(advanced_data)

selected_columns = advanced_data.columns[0:14].tolist() 
Q_Report = quality_report[quality_report['Column_Name'].isin(selected_columns)]
Q_Report

,Column_Name,Data_Type,Total_Rows,Missing_Values,Missing_Percentage,Blank_Values,Blank_Percentage,Zero_Values,Zero_Percentage,Duplicate_Values,Unique_Values
1,Boost Group,object,148,1,0.68,0,0.0,0,0.00,143,4
2,DM Times,object,148,1,0.68,0,0.0,0,0.00,142,5
0,Date,object,148,0,0.00,0,0.0,0,0.00,117,31
3,Name,object,148,0,0.00,0,0.0,0,0.00,121,27
4,Fans Gained,float64,148,0,0.00,0,0.0,1,0.68,24,124
5,Revenue,float64,148,0,0.00,0,0.0,1,0.68,0,148
6,Agency,object,148,0,0.00,0,0.0,0,0.00,145,3
7,Last Edited Time,datetime64[ns],148,0,0.00,0,0.0,0,0.00,16,132
8,Half,object,148,0,0.00,0,0.0,0,0.00,146,2
9,Start of Week,object,148,0,0.00,0,0.0,0,0.00,143,5


In [364]:
# Check common values for MODEL 13
model_13_data = advanced_data[advanced_data['Name'] == 'MODEL 13'][['Boost Group', 'DM Times']].mode()
print("Common values for MODEL 13:", model_13_data)

Common values for MODEL 13:   Boost Group   DM Times
0     Group B  8AM & 2PM


In [365]:
# Replace null 'Boost Group' values with "Group B" and 'DM Times' with "8AM & 2PM"
advanced_data['Boost Group'] = advanced_data['Boost Group'].fillna('Group B')
advanced_data['DM Times'] = advanced_data['DM Times'].fillna('8AM & 2PM')

In [366]:
### QUALITY REPORT (POST SECOND CLEANING)

def create_data_quality_report(df):
    # Initialize dictionary to store quality metrics
    quality_report = {
        'Column_Name': [],
        'Data_Type': [],
        'Total_Rows': [],
        'Missing_Values': [],
        'Missing_Percentage': [],
        'Blank_Values': [],
        'Blank_Percentage': [],
        'Zero_Values': [],
        'Zero_Percentage': [],
        'Duplicate_Values': [],
        'Unique_Values': []
    }
    
    total_rows = len(df)
    
    for column in df.columns:
        # Count missing values
        missing_count = df[column].isna().sum()
        
        # Count blank values (empty strings)
        blank_count = df[column].astype(str).str.strip().eq('').sum()
        
        # Count zero values for numeric columns
        if pd.api.types.is_numeric_dtype(df[column]):
            zero_count = (df[column] == 0).sum()
        else:
            zero_count = 0
            
        # Count unique values
        unique_count = df[column].nunique()
        
        # Add to report
        quality_report['Column_Name'].append(column)
        quality_report['Data_Type'].append(str(df[column].dtype))
        quality_report['Total_Rows'].append(total_rows)
        quality_report['Missing_Values'].append(missing_count)
        quality_report['Missing_Percentage'].append(round(missing_count/total_rows * 100, 2))
        quality_report['Blank_Values'].append(blank_count)
        quality_report['Blank_Percentage'].append(round(blank_count/total_rows * 100, 2))
        quality_report['Zero_Values'].append(zero_count)
        quality_report['Zero_Percentage'].append(round(zero_count/total_rows * 100, 2))
        quality_report['Duplicate_Values'].append(df[column].duplicated().sum())
        quality_report['Unique_Values'].append(unique_count)
    
    # Create DataFrame from the quality report
    quality_df = pd.DataFrame(quality_report)
    
    # Sort by missing percentage (descending)
    quality_df = quality_df.sort_values('Missing_Percentage', ascending=False)
    
    return quality_df

# Create the quality report
quality_report = create_data_quality_report(advanced_data)

selected_columns = advanced_data.columns[0:14].tolist() 
Q_Report = quality_report[quality_report['Column_Name'].isin(selected_columns)]
Q_Report

,Column_Name,Data_Type,Total_Rows,Missing_Values,Missing_Percentage,Blank_Values,Blank_Percentage,Zero_Values,Zero_Percentage,Duplicate_Values,Unique_Values
0,Date,object,148,0,0.0,0,0.0,0,0.00,117,31
1,Boost Group,object,148,0,0.0,0,0.0,0,0.00,144,4
2,DM Times,object,148,0,0.0,0,0.0,0,0.00,143,5
3,Name,object,148,0,0.0,0,0.0,0,0.00,121,27
4,Fans Gained,float64,148,0,0.0,0,0.0,1,0.68,24,124
5,Revenue,float64,148,0,0.0,0,0.0,1,0.68,0,148
6,Agency,object,148,0,0.0,0,0.0,0,0.00,145,3
7,Last Edited Time,datetime64[ns],148,0,0.0,0,0.0,0,0.00,16,132
8,Half,object,148,0,0.0,0,0.0,0,0.00,146,2
9,Start of Week,object,148,0,0.0,0,0.0,0,0.00,143,5


### Findings:

- Was found a missing value for a Boost Group related to MODEL 13, so this value is replaced with "Group B" and a DT Time "8AM & 2PM" for the same group and MODEL, taking as a reference the last value of the same group and MODEL.
- The missing `Boost Group` and `DM Times` for `MODEL 13` were filled with defaults (`Group B`, `8AM & 2PM`). Consider using the mode or most frequent values for `MODEL 13` for better context.
- One record in `Revenue` and `Fans Gained` has a zero value (0.68% of rows), potentially indicating missing or no activity for that day.
- `Revenue` has 100% unique values (148 unique out of 148 rows), indicating high variability or no aggregation at the daily level.
- Columns `Week` and `Month` were dropped from `advanced_data`. Consider documenting the rationale, as these could be useful for temporal analysis or alignment with `wow_data`.
- The dataset covers a limited date range, with only 5 unique weeks (e.g., `2024-09-29` to `2024-10-05`), restricting long-term trend analysis.

In [367]:
### Sending cleaning data to the folder '01_cleaned_data'

advanced_data.to_csv(r"C:\Users\joey_\Documents\Visual Code (Clone)\Proyectos\AM3 TEST\00_test_A\00_datasources\01_cleaned_data\advanced_data.csv", index=False)
print("Done")

Done
